In [1]:
import pandas as pd
from IPython.display import clear_output

# df = pd.read_csv("/kaggle/input/lora-df/data_for_lora.csv")
# df = pd.read_csv("/kaggle/input/data-for-lora-2/data_for_lora_2.csv")
# df = pd.read_csv("/kaggle/input/data-for-lora-3/data_for_lora_3.csv")
df = pd.read_csv("/kaggle/input/lora4-ygpt-only/data_for_lora_ygpt_only.csv")
df.head()

,text,label
0,"{$SBER} ..................⬆️300,0 вопрос тольк...",1.0
1,{$SBER} на чем летим?,0.0
2,"{$SBER} как прекрасен шортокрыл, посмотри....\...",-1.0
3,✅ 10 января Лукойл {$LKOH} рассмотрит итоги 20...,0.0
4,"{$SBER}\n \nЧто быстрее, скорость света, или с...",0.0


In [2]:
from sklearn.model_selection import train_test_split

X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

In [3]:
from datasets import Dataset, DatasetDict, Value

train_df = pd.DataFrame({'text': X_train, 'labels': y_train})
val_df = pd.DataFrame({'text': X_val, 'labels': y_val})
test_df = pd.DataFrame({'text': X_test, 'labels': y_test})

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# --- Проверка ---
print("\nСтруктура DatasetDict:")
print(dataset_dict)
print("\nПример записи из обучающего набора:")
print(dataset_dict['train'][4])


Структура DatasetDict:
DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 28342
    })
    validation: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 3150
    })
    test: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 3500
    })
})

Пример записи из обучающего набора:
{'text': '{$SBER} оно где-то рядом, смотрите во всех акциях РФ)', 'labels': 0.0, '__index_level_0__': 23574}


In [ ]:
def map_labels(example):
    example['labels'] = label_map[example['labels']]
    return example

label_map = {-1: 0, 0: 1, 1: 2}
reverse_label_map = {v: k for k, v in label_map.items()}
num_labels = len(label_map)

dataset_dict = dataset_dict.map(map_labels)

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
import torch

# Загружаем токенизатор от исходной модели
try:
    model_name = "tabularisai/multilingual-sentiment-analysis"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    clear_output()
    print("Токенизатор загружен!")
except Exception as e:
    raise e

Токенизатор загружен!


In [ ]:
# Функция токенизации
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False) # Паддинг будет позже с DataCollator

# Применяем токенизацию ко всем данным
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

# Удаляем ненужную колонку 'text', так как она уже преобразована в input_ids/attention_mask
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

print("Типы данных ДО кастинга:", dataset_dict['train'].features) # Посмотреть исходный тип
dataset_dict = dataset_dict.cast_column("labels", Value('int64'))
print("Типы данных ПОСЛЕ кастинга:", dataset_dict['train'].features)

# Устанавливаем формат для PyTorch
tokenized_datasets.set_format("torch")

# Data Collator для динамического паддинга батчей
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
from dataclasses import dataclass
from transformers import PreTrainedTokenizerBase
from transformers.utils import PaddingStrategy
from typing import Any, Dict, List, Optional, Union
import torch

@dataclass
class CustomDataCollatorWithPadding:
    """
    Кастомный Data Collator, который использует паддинг токенизатора
    и гарантирует, что 'labels' будут иметь тип torch.long.
    """
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    return_tensors: str = "pt" # Возвращаем PyTorch тензоры

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        # Извлекаем метки до обработки остальных признаков
        labels = None
        if "labels" in features[0].keys():
            labels = [feature["labels"] for feature in features] # Собираем метки

        # Используем токенизатор для паддинга input_ids, attention_mask и т.д.
        features_for_padding = [{k: v for k, v in feature.items() if k != 'labels'} for feature in features]
        batch = self.tokenizer.pad(
            features_for_padding,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )

        # Добавляем метки обратно в батч, ПРЕОБРАЗУЯ ИХ В TENSOR ТИПА LONG
        if labels is not None:
            # batch["labels"] = torch.tensor(labels, dtype=torch.long) # <<< Гарантируем torch.long
            batch["labels"] = torch.stack(labels)
            batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)

        return batch

custom_data_collator = CustomDataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# --- ЗАГРУЗКА БАЗОВОЙ МОДЕЛИ ---
# Важно: указываем новое количество классов (num_labels=3)
# и ignore_mismatched_sizes=True, чтобы игнорировать несовпадение размера
# выходного слоя классификатора (у предобученной модели 5 выходов, нам нужно 3).
# Это приведет к инициализации *нового* случайного слоя классификации поверх
# предобученных слоев DistilBERT.
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True,
    # Добавим маппинги id <-> label для удобства
    id2label={i: f"LABEL_{reverse_label_map[i]}" for i in range(num_labels)},
    label2id={f"LABEL_{reverse_label_map[i]}": i for i in range(num_labels)}
)

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

# --- КОНФИГУРАЦИЯ LoRA ---
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, # Тип задачи - классификация последовательностей
    r=16,                       # Ранг матриц адаптера (типичные значения: 8, 16, 32)
    lora_alpha=32,              # Коэффициент масштабирования (часто 2*r)
    lora_dropout=0.1,           # Dropout для LoRA слоев
    bias="none",                # Обычно не обучаем смещения в LoRA ('none' или 'all')
    # Указываем модули, к которым применяем LoRA.
    # Для DistilBERT это обычно 'q_lin' и 'v_lin' в слоях внимания.
    # Можно проверить названия командой: print(model)
    target_modules=["q_lin", "v_lin"],
)

# --- ПРИМЕНЕНИЕ LoRA К МОДЕЛИ ---
peft_model = get_peft_model(model, lora_config)

print("\nПараметры модели после применения LoRA:")
peft_model.print_trainable_parameters()

In [1]:
# !pip install -U evaluate

In [9]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

# # --- МЕТРИКИ ---
# # accuracy_metric = evaluate.load("accuracy")
# # f1_metric = evaluate.load("f1")

# # def compute_metrics(eval_pred):
# #     predictions, labels = eval_pred
# #     # Получаем предсказанный класс (индекс с максимальной логитом)
# #     predictions = np.argmax(predictions, axis=1)
    
# #     acc = accuracy_metric.compute(predictions=predictions, references=labels)
# #     f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted") # Используем weighted F1 для многоклассовой задачи
    
# #     return {
#         "accuracy": acc["accuracy"],
# #         "f1_weighted": f1["f1"],
# #     }

In [10]:
import wandb
from kaggle_secrets import UserSecretsClient # Импортируем клиент для доступа к секретам

# Получаем доступ к секретам пользователя
user_secrets = UserSecretsClient()

# Получаем значение секрета по его Label (имени), которое вы задали
# Убедитесь, что 'WANDB_API_KEY' точно совпадает с Label, который вы ввели на шаге 2
wandb_api_key = user_secrets.get_secret("wb_lora") 

# Логинимся, передавая ключ напрямую в функцию
wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ez3nx to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
import os
os.environ["WANDB_PROJECT"] = "sentiment_lora_finetuning"

In [ ]:
# --- АРГУМЕНТЫ ОБУЧЕНИЯ ---
# Настройте пути, гиперпараметры под свои нужды и ресурсы
output_dir = "./sentiment_lora_finetuned"
learning_rate = 2e-4 # LoRA часто требует бОльший learning rate, чем full fine-tuning
batch_size = 64
num_train_epochs = 5 # Обычно достаточно нескольких эпох для LoRA
weight_decay = 0.01

model_name_short = model_name.split('/')[-1] # "multilingual-sentiment-analysis"
run_name = f"{model_name_short}-lora-r{lora_config.r}-alpha{lora_config.lora_alpha}-lr{learning_rate}-epochs{num_train_epochs}"

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    label_names=["labels"],
    weight_decay=weight_decay,
    eval_strategy="epoch", # Оценивать после каждой эпохи
    save_strategy="epoch",       # Сохранять после каждой эпохи
    logging_strategy="epoch",    # Логировать после каждой эпохи
    load_best_model_at_end=True, # Загрузить лучшую модель в конце обучения
    metric_for_best_model="f1_weighted", # Метрика для выбора лучшей модели
    push_to_hub=False,           # Установите True, если хотите загрузить на Hugging Face Hub
    fp16=torch.cuda.is_available(), # Использовать смешанную точность, если доступен GPU
    # --- КЛЮЧЕВЫЕ ИЗМЕНЕНИЯ ДЛЯ W&B ---
    report_to="wandb",              # <--- Указываем W&B как платформу для логирования
    run_name=run_name,              # <--- Задаем имя запуска (отобразится в W&B UI)
    # ---------------------------------
    # Остальные аргументы по необходимости...
    logging_dir='./logs', 
)

In [ ]:
trainer = Trainer(
    model=peft_model,               # Используем PEFT модель
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
    data_collator=custom_data_collator,    # Для динамического паддинга
    compute_metrics=compute_metrics,  
)

In [ ]:
# !pip install --upgrade wandb transformers accelerate datasets peft

In [ ]:
# --- ЗАПУСК ОБУЧЕНИЯ ---
print("\nНачало обучения...")
trainer.train()

# --- СОХРАНЕНИЕ АДАПТЕРА ---
# Trainer автоматически сохранит лучший адаптер в output_dir/best_model
# Можно также сохранить явно последнюю версию адаптера:
adapter_path = f"{output_dir}/final_adapter"
peft_model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path) # Сохраним и токенизатор рядом
print(f"Обучение завершено. Финальный адаптер LoRA сохранен в: {adapter_path}")

In [ ]:
print(model)

In [ ]:
model_2 = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True,
    # Добавим маппинги id <-> label для удобства
    id2label={i: f"LABEL_{reverse_label_map[i]}" for i in range(num_labels)},
    label2id={f"LABEL_{reverse_label_map[i]}": i for i in range(num_labels)}
)

In [ ]:
lora_config_2 = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,                       
    lora_alpha=32,              
    lora_dropout=0.1,           
    bias="none",         
    target_modules=["q_lin", "v_lin", "out_lin", "lin1", "lin2"],
)

# --- ПРИМЕНЕНИЕ LoRA К МОДЕЛИ ---
peft_model_2 = get_peft_model(model_2, lora_config_2)

print("\nПараметры модели после применения LoRA:")
peft_model_2.print_trainable_parameters()

In [ ]:
# Загружаем необходимые метрики из evaluate
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision") # Добавили Precision
recall_metric = evaluate.load("recall")     # Добавили Recall

def compute_metrics(eval_pred):
    """
    Вычисляет accuracy, weighted precision, recall и F1 для многоклассовой задачи.
    """

    predictions_logits, labels = eval_pred
    # Получаем предсказанный класс (индекс с максимальной логитом)
    predictions = np.argmax(predictions_logits, axis=1)

    # Рассчитываем метрики
    acc = accuracy_metric.compute(predictions=predictions, references=labels)

    # Используем average="weighted" для Precision, Recall, F1
    # чтобы учесть количество примеров в каждом классе
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")

    # Возвращаем словарь с метриками
    return {
        "accuracy": acc["accuracy"],
        "precision_weighted": precision["precision"], # Добавили ключ для precision
        "recall_weighted": recall["recall"],       # Добавили ключ для recall
        "f1_weighted": f1["f1"],
    }


In [ ]:
output_dir = "./sentiment_lora_finetuned_2"
learning_rate = 1e-4 # LoRA часто требует бОльший learning rate, чем full fine-tuning
batch_size = 32
num_train_epochs = 5 # Обычно достаточно нескольких эпох для LoRA
weight_decay = 0.001

model_name_short = model_name.split('/')[-1] # "multilingual-sentiment-analysis"
run_name = f"{model_name_short}-lora-r{lora_config.r}-alpha{lora_config.lora_alpha}-lr{learning_rate}-epochs{num_train_epochs}"

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=learning_rate,          # Начальная скорость обучения
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    label_names=["labels"],
    weight_decay=weight_decay,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    push_to_hub=False,
    fp16=torch.cuda.is_available(),

    # --- ДОБАВЛЕННЫЕ ПАРАМЕТРЫ ШЕДУЛЕРА ---
    lr_scheduler_type='cosine',      # Тип планировщика: косинусный
    warmup_ratio=0.1,                # Доля шагов для прогрева (10% от общих шагов обучения)
    # Или можно использовать warmup_steps=N, если вы знаете точное число шагов прогрева (N)
    # ---------------------------------------

    # --- Параметры W&B ---
    report_to="wandb",
    run_name=run_name,
    logging_dir='./logs',
)

In [ ]:
trainer = Trainer(
    model=peft_model_2,               # Используем PEFT модель
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
    data_collator=custom_data_collator,    # Для динамического паддинга
    compute_metrics=compute_metrics,  
)

In [ ]:
# --- ЗАПУСК ОБУЧЕНИЯ ---
print("\nНачало обучения...")
trainer.train()

# --- СОХРАНЕНИЕ АДАПТЕРА ---
# Trainer автоматически сохранит лучший адаптер в output_dir/best_model
# Можно также сохранить явно последнюю версию адаптера:
adapter_path = f"{output_dir}/final_adapter"
peft_model_2.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path) # Сохраним и токенизатор рядом
print(f"Обучение завершено. Финальный адаптер LoRA сохранен в: {adapter_path}")

In [ ]:
model_3 = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True,
    # Добавим маппинги id <-> label для удобства
    id2label={i: f"LABEL_{reverse_label_map[i]}" for i in range(num_labels)},
    label2id={f"LABEL_{reverse_label_map[i]}": i for i in range(num_labels)}
)

from peft import LoraConfig, TaskType, get_peft_model

lora_config_3 = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=64,                       
    lora_alpha=128,              
    lora_dropout=0.1,           
    bias="none",         
    target_modules=["q_lin", "v_lin", "k_lin", "out_lin", "lin1", "lin2"],
)

# --- ПРИМЕНЕНИЕ LoRA К МОДЕЛИ ---
peft_model_3 = get_peft_model(model_3, lora_config_3)

print("\nПараметры модели после применения LoRA:")
peft_model_3.print_trainable_parameters()

In [14]:
output_dir = "./sentiment_lora_finetuned_BIG_SET"
learning_rate = 2e-4 # LoRA часто требует бОльший learning rate, чем full fine-tuning
batch_size = 32
num_train_epochs = 7 # Обычно достаточно нескольких эпох для LoRA
weight_decay = 0.001
gradient_accumulation_steps = 4

model_name_short = model_name.split('/')[-1] # "multilingual-sentiment-analysis"
run_name = f"{model_name_short}-lora-r{lora_config_3.r}-alpha{lora_config_3.lora_alpha}-lr{learning_rate}-epochs{num_train_epochs}"

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=learning_rate,          # Начальная скорость обучения
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    label_names=["labels"],
    weight_decay=weight_decay,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    push_to_hub=False,
    fp16=torch.cuda.is_available(),
    lr_scheduler_type='cosine',
    warmup_ratio=0.2,
    # --- ДОБАВЛЕННЫЙ ПАРАМЕТР ---
    gradient_accumulation_steps=gradient_accumulation_steps, # Указываем шаги аккумуляции
    # --------------------------
    report_to="wandb",
    run_name=run_name,
    logging_dir='./logs',
)

In [15]:
trainer = Trainer(
    model=peft_model_3,               # Используем PEFT модель
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
    data_collator=custom_data_collator,    # Для динамического паддинга data_collator
    compute_metrics=compute_metrics,  
)

In [16]:
# --- ЗАПУСК ОБУЧЕНИЯ ---
print("\nНачало обучения...")
trainer.train()
wandb.finish()
# --- СОХРАНЕНИЕ АДАПТЕРА ---
# Trainer автоматически сохранит лучший адаптер в output_dir/best_model
# Можно также сохранить явно последнюю версию адаптера:
adapter_path = f"{output_dir}/final_adapter"
peft_model_3.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path) # Сохраним и токенизатор рядом
print(f"Обучение завершено. Финальный адаптер LoRA сохранен в: {adapter_path}")


Начало обучения...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted
0,0.953000,0.823498,0.637460,0.640601,0.637460,0.636775
1,0.777600,0.722078,0.689206,0.690140,0.689206,0.689331
2,0.673300,0.700395,0.702222,0.705085,0.702222,0.702327
3,0.594000,0.690893,0.707937,0.710926,0.707937,0.707477
4,0.526600,0.699082,0.717460,0.718003,0.717460,0.717522
5,0.471500,0.718486,0.716190,0.717527,0.716190,0.716153
6,0.442600,0.728787,0.713016,0.713090,0.713016,0.713032


/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instea

eval/accuracy,▁▆▇▇███
eval/f1_weighted,▁▆▇▇███
eval/loss,█▃▂▁▁▂▃
eval/precision_weighted,▁▅▇▇███
eval/recall_weighted,▁▆▇▇███
eval/runtime,█▇▁▁▂▁▂
eval/samples_per_second,▁▂██▇█▇
eval/steps_per_second,▁▂██▇█▇
train/epoch,▁▁▂▂▃▃▄▄▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███
train/grad_norm,▁▂▂▂█▅▄


Обучение завершено. Финальный адаптер LoRA сохранен в: ./sentiment_lora_finetuned_BIG_SET/final_adapter


In [20]:
import numpy as np
import torch
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)

# --- Убедитесь, что эти объекты существуют из вашего кода обучения LoRA ---
# tokenized_datasets["validation"] - ваш валидационный датасет, токенизированный,
#                                     с колонкой 'labels' (значения 0, 1, 2)
# label_map = {-1: 0, 0: 1, 1: 2} # Ваше отображение
# reverse_label_map = {0: -1, 1: 0, 2: 1} # Ваше обратное отображение
# -----------------------------------------------------------------------


# --- 1. Определяем функцию метрик для ОРИГИНАЛЬНОЙ модели ---
#    Она будет мапить 5 предсказанных классов в 3

# Определяем маппинг: индекс оригинального класса -> индекс нашего класса (0, 1, 2)
original_5_to_3_map = {
    0: label_map[-1], # Very Negative -> Наш Negative (0)
    1: label_map[-1], # Negative      -> Наш Negative (0)
    2: label_map[0],  # Neutral       -> Наш Neutral (1)
    3: label_map[1],  # Positive      -> Наш Positive (2)
    4: label_map[1]   # Very Positive -> Наш Positive (2)
}
print(f"Маппинг 5->3 классов: {original_5_to_3_map}")

def compute_metrics_original_model(eval_pred):
    """
    Вычисляет метрики для 3 классов, получая на вход логиты для 5 классов
    и истинные метки для 3 классов.
    """
    try:
        logits_5_class, labels_3_class = eval_pred # labels_3_class уже 0, 1, 2

        # Получаем предсказанные классы из 5 логитов (индексы 0-4)
        predictions_5_class_indices = np.argmax(logits_5_class, axis=1)

        # Маппим предсказания 5 классов в наши 3 класса (индексы 0-2)
        predictions_3_class_mapped = np.array(
            [original_5_to_3_map[p_5_idx] for p_5_idx in predictions_5_class_indices]
        )

        # Вычисляем метрики, сравнивая смапленные 3-класс. предсказания с 3-класс. метками
        acc = accuracy_metric.compute(predictions=predictions_3_class_mapped, references=labels_3_class)
        precision = precision_metric.compute(predictions=predictions_3_class_mapped, references=labels_3_class, average="weighted")
        recall = recall_metric.compute(predictions=predictions_3_class_mapped, references=labels_3_class, average="weighted")
        f1 = f1_metric.compute(predictions=predictions_3_class_mapped, references=labels_3_class, average="weighted")

        return {
            "accuracy": acc["accuracy"],
            "precision_weighted": precision["precision"],
            "recall_weighted": recall["recall"],
            "f1_weighted": f1["f1"],
        }
    except Exception as e:
        print(f"!!! Ошибка в compute_metrics_original_model: {e} !!!")
        return {"metric_computation_error": 1}


# --- 2. Загрузка ОРИГИНАЛЬНОЙ модели и токенизатора ---
model_name_original = "tabularisai/multilingual-sentiment-analysis"
print(f"\nЗагрузка ОРИГИНАЛЬНОЙ 5-классовой модели {model_name_original}...")
# Загружаем как есть, с 5 классами по умолчанию
original_model = AutoModelForSequenceClassification.from_pretrained(model_name_original)
original_tokenizer = AutoTokenizer.from_pretrained(model_name_original)
print("Оригинальная модель загружена.")


# --- 3. Подготовка к оценке ---
# !!! ВАЖНО: Убедитесь, что ваш `tokenized_datasets["validation"]` был токенизирован
#           с помощью ТОГО ЖЕ `original_tokenizer`. Если нет, его нужно перетокенизировать!
# Пример перетокенизации, если нужно (замените dataset_dict['validation'] на ваш исходный валидационный датасет):
# print("Перетокенизация валидационного набора оригинальным токенизатором...")
# validation_dataset_retokenized = dataset_dict['validation'].map(
#     lambda ex: original_tokenizer(ex["text"], truncation=True, padding=False, max_length=512), batched=True
# )
# validation_dataset_retokenized = validation_dataset_retokenized.remove_columns(["text"]) # и другие ненужные колонки
# # Убедимся, что метки 'labels' типа int64
# validation_dataset_retokenized = validation_dataset_retokenized.cast_column("labels", Value('int64'))
# validation_dataset_retokenized.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])
# eval_dataset_for_original = validation_dataset_retokenized
# print("Перетокенизация завершена.")

# Если вы уверены, что токенизация была одинаковой:
eval_dataset_for_original = tokenized_datasets["validation"]


# Используем минимальные аргументы для оценки
eval_args = TrainingArguments(
    output_dir="./eval_original_model_temp", # Просто временная папка
    per_device_eval_batch_size=32,        # Можно увеличить для скорости
    report_to="none",                     # Не логируем эту оценку
)

# Создаем Trainer для ОЦЕНКИ оригинальной модели
eval_trainer_original = Trainer(
    model=original_model,                   # <<< ОРИГИНАЛЬНАЯ модель
    args=eval_args,
    eval_dataset=eval_dataset_for_original, # <<< Ваш валидационный набор
    processing_class=original_tokenizer,           # <<< ОРИГИНАЛЬНЫЙ токенизатор
    data_collator=custom_data_collator,
    compute_metrics=compute_metrics_original_model # <<< СПЕЦИАЛЬНАЯ функция метрик
)

# --- 4. Запуск оценки ---
print("\nЗапуск оценки ОРИГИНАЛЬНОЙ модели на валидационном наборе...")
original_model_metrics = eval_trainer_original.evaluate()

print("\n--- Метрики ОРИГИНАЛЬНОЙ модели (преобразованные к 3 классам) ---")
for key, value in original_model_metrics.items():
    print(f"{key}: {value:.6f}")

# --- 5. Сравнение (если у вас есть метрики последней LoRA модели) ---
# Предположим, лучшие метрики вашей LoRA модели хранятся в словаре best_lora_metrics
wandb.init()
best_lora_metrics = trainer.evaluate() # Если Trainer еще доступен и содержит лучшую модель
# или загрузите из логов W&B/файла

print("\n--- Сравнение F1 Weighted ---")
print(f"Оригинальная модель: {original_model_metrics.get('eval_f1_weighted', 'N/A'):.6f}")
print(f"LoRA модель (лучшая): {best_lora_metrics.get('eval_f1_weighted', 'N/A'):.6f}")

Маппинг 5->3 классов: {0: 0, 1: 0, 2: 1, 3: 2, 4: 2}

Загрузка ОРИГИНАЛЬНОЙ 5-классовой модели tabularisai/multilingual-sentiment-analysis...
Оригинальная модель загружена.

Запуск оценки ОРИГИНАЛЬНОЙ модели на валидационном наборе...


/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



--- Метрики ОРИГИНАЛЬНОЙ модели (преобразованные к 3 классам) ---
eval_loss: 1.645286
eval_accuracy: 0.447302
eval_precision_weighted: 0.447048
eval_recall_weighted: 0.447302
eval_f1_weighted: 0.443735
eval_runtime: 29.016000
eval_samples_per_second: 108.561000
eval_steps_per_second: 1.723000


/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



--- Сравнение F1 Weighted ---
Оригинальная модель: 0.443735
LoRA модель (лучшая): 0.717522


In [21]:
best_lora_metrics

{'eval_loss': 0.6990819573402405,
 'eval_accuracy': 0.7174603174603175,
 'eval_precision_weighted': 0.7180032833786785,
 'eval_recall_weighted': 0.7174603174603175,
 'eval_f1_weighted': 0.717522136966659,
 'eval_runtime': 35.2164,
 'eval_samples_per_second': 89.447,
 'eval_steps_per_second': 1.42,
 'epoch': 6.993227990970655}

In [22]:
original_model_metrics

{'eval_loss': 1.6452864408493042,
 'eval_accuracy': 0.4473015873015873,
 'eval_precision_weighted': 0.4470480131892384,
 'eval_recall_weighted': 0.4473015873015873,
 'eval_f1_weighted': 0.4437350351888902,
 'eval_runtime': 29.016,
 'eval_samples_per_second': 108.561,
 'eval_steps_per_second': 1.723}

In [31]:
from huggingface_hub import notebook_login
notebook_login() # Попросит ввести ваш токен доступа (из настроек HF)

In [29]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel
import torch

base_model_name = "tabularisai/multilingual-sentiment-analysis"
adapter_path = "/kaggle/working/sentiment_lora_finetuned_BIG_SET/final_adapter" # Укажите правильный путь к папке с лучшим адаптером
num_labels = 3
# ... (определите reverse_label_map_inference) ...

# Загружаем базу
base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True,
    id2label={i: f"LABEL_{reverse_label_map[i]}" for i in range(num_labels)},
    label2id={f"LABEL_{reverse_label_map[i]}": i for i in range(num_labels)}
)

# Применяем адаптер
inference_model = PeftModel.from_pretrained(base_model, adapter_path)

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

# Убедитесь, что у токенизатора установлен pad_token, если он нужен
if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
     tokenizer.pad_token_id = tokenizer.eos_token_id

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at tabularisai/multilingual-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
# Задайте имя репозитория на Hugging Face Hub
# Рекомендуется формат: "your-username/model-name-description"
repo_id = "ez3nx/multilingual-sentiment-3class-lora-adapter" # <<< ЗАМЕНИТЕ НА СВОЕ

print(f"Публикация адаптера LoRA и токенизатора в репозиторий: {repo_id}")

# Пушим адаптер (PeftModel сам знает, что пушить только адаптер)
inference_model.push_to_hub(repo_id)

# Пушим токенизатор в тот же репозиторий
tokenizer.push_to_hub(repo_id)

print("Публикация завершена.")

Публикация адаптера LoRA и токенизатора в репозиторий: ez3nx/multilingual-sentiment-3class-lora-adapter


adapter_model.safetensors:   0%|          | 0.00/23.6M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Публикация завершена.


In [16]:
# --- ЗАПУСК ОБУЧЕНИЯ ---
print("\nНачало обучения...")
trainer.train()
wandb.finish()
# --- СОХРАНЕНИЕ АДАПТЕРА ---
# Trainer автоматически сохранит лучший адаптер в output_dir/best_model
# Можно также сохранить явно последнюю версию адаптера:
adapter_path = f"{output_dir}/final_adapter"
peft_model_3.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path) # Сохраним и токенизатор рядом
print(f"Обучение завершено. Финальный адаптер LoRA сохранен в: {adapter_path}")


Начало обучения...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted
1,0.927700,0.810142,0.642426,0.645795,0.642426,0.639911
2,0.759400,0.735825,0.671240,0.676999,0.671240,0.670726
3,0.676600,0.711319,0.691548,0.691536,0.691548,0.691435
4,0.604100,0.710757,0.697311,0.701033,0.697311,0.697011
5,0.520100,0.721250,0.702525,0.703044,0.702525,0.702565


/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instea

eval/accuracy,▁▄▇▇██
eval/f1_weighted,▁▄▇▇██
eval/loss,█▃▁▁▁▂
eval/precision_weighted,▁▅▇███
eval/recall_weighted,▁▄▇▇██
eval/runtime,▂▁▂▅█▁
eval/samples_per_second,▇█▇▄▁█
eval/steps_per_second,▇██▄▁█
train/epoch,▁▁▂▂▄▄▅▅▇▇███
train/global_step,▁▁▂▂▄▄▅▅▇▇███
train/grad_norm,▂▂▅▁▅█


Обучение завершено. Финальный адаптер LoRA сохранен в: ./sentiment_lora_finetuned_BIG_SET/final_adapter


## ПОЛНЫЙ ФАЙНТЮН

In [44]:
import pandas as pd
from datasets import Dataset, DatasetDict, Value
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding, # Используем стандартный коллатор
    TrainingArguments,
    Trainer,
    TrainerCallback # Импортируем TrainerCallback
)
from transformers.integrations import WandbCallback # Если вы используете W&B
import torch
import numpy as np
import evaluate
import os
import wandb # Если вы используете W&B



# --- Установка формата PyTorch (Как раньше) ---
columns_to_set_format = ['input_ids', 'attention_mask', 'labels']
tokenized_datasets.set_format("torch", columns=columns_to_set_format)

# --- 3. Data Collator (Используем СТАНДАРТНЫЙ) ---
# Если ошибка типа float32 вернется, можно раскомментировать и использовать
# ваш CustomDataCollatorWithPadding из предыдущих шагов.
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
custom_data_collator = CustomDataCollatorWithPadding(tokenizer=tokenizer) # Опционально

# --- 4. Загрузка МОДЕЛИ (Без PEFT/LoRA) ---
# Загружаем базовую модель С НУЖНЫМ КОЛИЧЕСТВОМ КЛАССОВ
# ignore_mismatched_sizes=True автоматически заменит голову классификатора
print(f"Загрузка базовой модели {model_name} для Full Fine-Tuning...")
model_fft = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,              # <<< 3 класса
    ignore_mismatched_sizes=True,     # <<< Заменить голову классификатора
    id2label={i: f"LABEL_{reverse_label_map[i]}" for i in range(num_labels)},
    label2id={f"LABEL_{reverse_label_map[i]}": i for i in range(num_labels)}
)
print("Модель загружена.")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at tabularisai/multilingual-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Загрузка базовой модели tabularisai/multilingual-sentiment-analysis для Full Fine-Tuning...
Модель загружена.


In [45]:
# Проверка количества обучаемых параметров (должно быть почти все)
total_params = sum(p.numel() for p in model_fft.parameters())
trainable_params = sum(p.numel() for p in model_fft.parameters() if p.requires_grad)
print(f"Всего параметров: {total_params:,}")
print(f"Обучаемых параметров: {trainable_params:,} ({(trainable_params/total_params)*100:.2f}%)")

Всего параметров: 135,326,979
Обучаемых параметров: 135,326,979 (100.00%)


In [46]:
# --- 6. Аргументы обучения (Изменяем learning_rate и output_dir/run_name) ---
output_dir = "./sentiment_fft_finetuned" # Новая директория для FFT
logging_dir = './logs_fft'             # Новая директория логов

# !!! ВАЖНО: Низкий learning rate для FFT !!!
learning_rate = 2e-5  # Типичное значение для FFT (попробуйте 2e-5, 5e-5 если нужно)

batch_size = 16 # Оставьте или измените в зависимости от памяти GPU
num_train_epochs = 6 # FFT часто требует меньше эпох, начните с 3-5
weight_decay = 0.0004
gradient_accumulation_steps = 4

# Настройка W&B (если используется)
os.environ["WANDB_PROJECT"] = "sentiment_fft_finetuning" # Новый проект или то же имя
model_name_short = model_name.split('/')[-1]
run_name = f"{model_name_short}-fft-lr{learning_rate}-epochs{num_train_epochs}" # Имя для FFT

training_args = TrainingArguments(
    output_dir=output_dir,
    logging_dir=logging_dir,
    learning_rate=learning_rate,          # <<< НОВЫЙ LR
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size, # Можно увеличить для ускорения оценки
    num_train_epochs=num_train_epochs,
    label_names=["labels"],
    weight_decay=weight_decay,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_weighted",
    push_to_hub=True,
    gradient_accumulation_steps=gradient_accumulation_steps,
    fp16=torch.cuda.is_available(),
    # --- Шедулер (оставляем, полезен для FFT) ---
    lr_scheduler_type='cosine',
    warmup_ratio=0.2,
    # --- W&B ---
    report_to="wandb",                  # или "none"
    run_name=run_name,
)

# --- 7. Инициализация Trainer (Передаем базовую модель) ---
# callbacks_list = []
# if training_args.report_to == "wandb":
#      # Убедитесь, что выполнен вход через wandb.login(key=...)
#     callbacks_list.append(WandbCallback())

In [47]:
trainer = Trainer(
    model=model_fft,                     # <<< ПЕРЕДАЕМ БАЗОВУЮ МОДЕЛЬ
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=custom_data_collator,      # <<< СТАНДАРТНЫЙ КОЛЛАТОР (или кастомный, если нужно)
    compute_metrics=compute_metrics,
    # callbacks=callbacks_list          # Передаем коллбэки (для W&B)
)

# --- 8. Обучение ---
print("\nНачало ПОЛНОГО Fine-Tuning...")
print("ПРЕДУПРЕЖДЕНИЕ: Это потребует значительно больше памяти GPU и времени, чем LoRA.")
trainer.train()

# --- 9. Сохранение (Trainer сам сохранит лучшую модель) ---
# Trainer автоматически сохранит лучший чекпоинт в training_args.output_dir
# благодаря save_strategy="epoch" и load_best_model_at_end=True.
# Сохранять вручную не обязательно, но можно сохранить финальную модель:
# final_model_path = f"{output_dir}/final_model"
# trainer.save_model(final_model_path)
# tokenizer.save_pretrained(final_model_path)
print(f"Обучение завершено. Лучшая модель сохранена в: {output_dir}")
# Если использовали W&B, не забудьте wandb.finish() если нужно

/tmp/ipykernel_31/2844417089.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Начало ПОЛНОГО Fine-Tuning...
ПРЕДУПРЕЖДЕНИЕ: Это потребует значительно больше памяти GPU и времени, чем LoRA.


/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted
1,0.951600,0.809786,0.646984,0.650579,0.646984,0.647044
2,0.749900,0.720034,0.690159,0.701094,0.690159,0.690040
3,0.628100,0.694548,0.713651,0.714047,0.713651,0.713595
4,0.537900,0.704709,0.716825,0.718316,0.716825,0.716716
5,0.434000,0.730151,0.716825,0.717263,0.716825,0.716870


/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instea

Обучение завершено. Лучшая модель сохранена в: ./sentiment_fft_finetuned


In [ ]:
# --- 10. Инференс (Загружаем модель из папки с результатом) ---
from transformers import pipeline

print("\n--- Инференс с дообученной моделью (FFT) ---")
# Загружаем модель, сохраненную Trainer'ом
best_model_path = output_dir # Trainer сохраняет лучшую модель в output_dir при load_best_model_at_end=True
# Если load_best_model_at_end=False, лучший чекпоинт будет в output_dir/checkpoint-XYZ

# Вариант 1: Использование pipeline
pipe = pipeline("text-classification", model=best_model_path, device=0 if torch.cuda.is_available() else -1)

# Вариант 2: Ручная загрузка и предсказание
# inference_tokenizer = AutoTokenizer.from_pretrained(best_model_path)
# inference_model = AutoModelForSequenceClassification.from_pretrained(best_model_path)
# inference_model.to("cuda" if torch.cuda.is_available() else "cpu")
# inference_model.eval()
# def predict_sentiment_fft(text):
#     inputs = inference_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
#     inputs = {k: v.to(inference_model.device) for k, v in inputs.items()}
#     with torch.no_grad():
#         outputs = inference_model(**inputs)
#         logits = outputs.logits
#     probabilities = torch.nn.functional.softmax(logits, dim=-1)
#     predicted_class_id = torch.argmax(probabilities, dim=-1).item()
#     predicted_label = reverse_label_map[predicted_class_id]
#     predicted_probability = probabilities[0, predicted_class_id].item()
#     return {"label": predicted_label, "score": predicted_probability, "label_id": predicted_class_id}

test_sentence_1 = "Это было невероятно хорошо, я в восторге!"
test_sentence_2 = "Фильм как фильм, ничего особенного."
test_sentence_3 = "Мне совсем не зашло, пустая трата времени."

# Используем pipeline
result1 = pipe(test_sentence_1)
result2 = pipe(test_sentence_2)
result3 = pipe(test_sentence_3)

# # Используем ручную функцию
# result1 = predict_sentiment_fft(test_sentence_1)
# result2 = predict_sentiment_fft(test_sentence_2)
# result3 = predict_sentiment_fft(test_sentence_3)


print(f"Предсказание для: '{test_sentence_1}' => {result1}")
print(f"Предсказание для: '{test_sentence_2}' => {result2}")
print(f"Предсказание для: '{test_sentence_3}' => {result3}")

In [16]:
torch.cuda.empty_cache()

import gc
gc.collect()
gc.collect()

0

In [17]:
# --- ЗАПУСК ОБУЧЕНИЯ ---
print("\nНачало обучения...")
trainer.train()
wandb.finish()
# --- СОХРАНЕНИЕ АДАПТЕРА ---
# Trainer автоматически сохранит лучший адаптер в output_dir/best_model
# Можно также сохранить явно последнюю версию адаптера:
adapter_path = f"{output_dir}/final_adapter"
peft_model_3.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path) # Сохраним и токенизатор рядом
print(f"Обучение завершено. Финальный адаптер LoRA сохранен в: {adapter_path}")


Начало обучения...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted
1,0.955100,0.859144,0.617865,0.625604,0.617865,0.617284
2,0.806600,0.782680,0.661002,0.667788,0.661002,0.660280
3,0.724500,0.769324,0.676688,0.679926,0.676688,0.675447
4,0.664500,0.750470,0.681046,0.685300,0.681046,0.681192
5,0.642000,0.752278,0.687146,0.687830,0.687146,0.687253


/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/tmp/ipykernel_31/2557029147.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instea

eval/accuracy,▁▅▇▇█
eval/f1_weighted,▁▅▇▇█
eval/loss,█▃▂▁▁
eval/precision_weighted,▁▆▇██
eval/recall_weighted,▁▅▇▇█
eval/runtime,▂▁▃█▁
eval/samples_per_second,▇█▆▁█
eval/steps_per_second,▇█▆▁█
train/epoch,▁▁▃▃▅▅▆▆███
train/global_step,▁▁▃▃▅▅▆▆███
train/grad_norm,▁▁▄█▇


Обучение завершено. Финальный адаптер LoRA сохранен в: ./sentiment_lora_finetuned_BIG_SET/final_adapter
